# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770870744849                   -0.52    9.0         
  2   -2.772146813269       -2.89       -1.33    1.0    104ms
  3   -2.772170127573       -4.63       -2.41    1.0    120ms
  4   -2.772170635988       -6.29       -3.10    1.0    126ms
  5   -2.772170722824       -7.06       -4.62    2.0    122ms
  6   -2.772170723001       -9.75       -4.78    2.0    148ms
  7   -2.772170723014      -10.89       -5.78    1.0    112ms
  8   -2.772170723015      -11.95       -6.54    2.0    128ms
  9   -2.772170723015      -13.78       -7.01    1.0    133ms
 10   -2.772170723015      -13.99       -8.10    2.0    137ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770701419822                   -0.53    9.0         
  2   -2.772055725421       -2.87       -1.31    1.0    132ms
  3   -2.7

1.7735580782557074

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770821345226                   -0.53    9.0         
  2   -2.772059691366       -2.91       -1.31    1.0    105ms
  3   -2.772083089993       -4.63       -2.57    1.0    107ms
  4   -2.772083388200       -6.53       -3.56    1.0    157ms
  5   -2.772083416202       -7.55       -3.91    2.0    124ms
  6   -2.772083417764       -8.81       -5.12    1.0    112ms
  7   -2.772083417809      -10.35       -5.45    2.0    150ms
  8   -2.772083417810      -11.90       -6.11    1.0    116ms
  9   -2.772083417811      -12.64       -6.64    2.0    145ms
 10   -2.772083417811   +  -14.45       -7.41    1.0    125ms
 11   -2.772083417811      -13.85       -7.95    1.0    119ms
 12   -2.772083417811   +  -13.82       -8.29    1.0    136ms

Polarizability via ForwardDiff:       1.7725349707912879
Polarizability via finite difference: 1.7735580782557074
